# 데이터 분석

## 4. 웹 크롤링으로 데이터 수집 - 정적 웹크롤링

### #그래프에서 한글사용하는 방법
- **(코랩에서)한글폰트 설치하기**

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 코랩에서 위 코드를 실행시킨 후  반드시 코랩 메뉴: "런타임>런타임 다시 시작" 합니다.

- **한글 폰트 지정하기**

In [ ]:
# 코랩에서 한글 폰트 종류와 이름이 win과 다를 수 있다!!!
# 코랩: NanumGothic, 윈도우: Malgun Gothic
import matplotlib.pyplot as plt
plt.rcParams.update({'font.family': 'Malgun Gothic',
                     'font.size': 12,
                     'figure.figsize': (6, 4),
                     'axes.unicode_minus':  False }) # 폰트 설정

### 1) 웹 크롤링 기초

### 라이브러리 설치하기

In [ ]:
# 정적 크롤링을 위한 requests 설치
!pip install requests

In [ ]:
# HTML과 XML 문서를 파싱하기 위한 파이썬 패키지
!pip install beautifulsoup4

In [ ]:
# 동적 크롤링을 위한 셀레니움 설치
!pip install selenium

In [ ]:
# 동적 크롤링을 위한 크롬드라이버 자동설치 라이브러리 설치
!pip install chromedriver-autoinstaller

In [ ]:
import requests
import bs4
import selenium
print(requests.__version__)
print(bs4.__version__)
print(selenium.__version__)

### 2) 정적 크롤링(스크래핑)

#### 1.웹 페이지 가져오기

In [ ]:
import requests

requests.get("https://www.naver.com")

In [ ]:
import requests

response = requests.get("https://google.com")

# 응답 상태
print('#응답 상태: ', response.status_code)

# 응답 바이너리 원문
print('#응답 바이너리 원문: ', response.content)

# 응답 UTF-8로 인코딩된 문자열
print('#응답 UTF-8로 인코딩된 문자열: ', response.text)

# 응답 헤더
print('#응답 헤더: ', response.headers)

# 응답 헤더: 콘텐트 유형
print('#응답 헤더유형: ', response.headers['Content-Type'])

#### 2.웹 페이지에서 정보 추출하기
- BeautifulSoup 라이브러리 사용하여 웹페이지에서 정보 추출하기
- 네이버 메인(https://www.naver.com/) > 검색어 입력(**눈물의여왕**) > 검색된 결과 페이지에서 **제목**만 추출

#### [예제] : Text 제목 정보 1개 가져오기 in (정적인 페이지)

* 뉴스 제목이 있는 위치 찾는 방법
    1. 크롬브라우저를 연다.
    2. Target URL =https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EB%88%88%EB%AC%BC%EC%9D%98%EC%97%AC%EC%99%95 페이지를 연결한다.
    3. 키보드에서 F12 키를 눌러 크롬 개발자 도구를 연다
    4. 개발자 도구에서 마우스로 영역선택( )메뉴를 클릭하고 마우스를 뉴스 제목 부분에 놓고 클릭한다.
    5.마우스 오른쪽 버튼을 클릭하고 메뉴(Copy > copy selector)를 선택한다.
    6. 5에서 복사된 뉴스 제목이 있는 영역으 tag를 코드에 붙여넣는다.


In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EB%88%88%EB%AC%BC%EC%9D%98%EC%97%AC%EC%99%95'
print(url)

response = requests.get(url)

if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    title = soup.select_one('#sp_nws1 > div.news_wrap.api_ani_send > div > div.news_contents > a.news_tit')
    print(title)
    print(f"제목: {title.attrs['title']}")  # 제목 추출 : title.get_text()
else :
    print(response.status_code)
print('----------')

#### [예제] Text 정보 여러 개 가져오기 in (정적인 페이지)

In [ ]:
import requests
from bs4 import BeautifulSoup

print('페이지 정보 추출하기-----')
url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EB%88%88%EB%AC%BC%EC%9D%98%EC%97%AC%EC%99%95'
print(url)

response = requests.get(url)

t_list, d_list, link_list = [], [], []
if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    ul = soup.select_one('ul.list_news') #공백에 있을 경우 . 사용
    # 제목, 링크
    titles = ul.select('li > div > div > div.news_contents > a.news_tit')
    for title in titles:
        t_list.append(title.attrs['title'])
        link_list.append(title.attrs['href'])
#         print(title.get_text())
    # 상세 설명
    titles = ul.select('li > div > div > div.news_contents > div.news_dsc > div.dsc_wrap > a')
    for title in titles:
        d_list.append(title.get_text())
else :
    print(response.status_code)
print('페이지 정보 추출완료-----')
t_list, d_list, link_list

In [ ]:
# pandas DataFrame으로 나타내기
import pandas as pd

data = {'title': t_list, 'desc':d_list,'link':link_list}
df = pd.DataFrame(data)

df.to_csv('my_naver_news.csv', index=False) # 파일로 저장하기

df.head()

#### **[실습] : 할리스 커피매장 정보 수집하기**
 - 매장정보 : https://www.hollys.co.kr/store/korea/korStore2.do?pageNo=2&sido=&gugun=&store=


In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import datetime

MAX = 100    # 총 가져올 건수
FILE = './data/hollys_매장정보.csv'

#[CODE 1]
def hollys_store(result):
    cnt, MAX_flag = 0, False
    for page in range(1,59):
#         Hollys_url = 'https://www.hollys.co.kr/store/korea/korStore.do?pageNo=%d&sido=&gugun=&store=' %page
        Hollys_url = 'https://www.hollys.co.kr/store/korea/korStore2.do?pageNo=%d&sido=&gugun=&store=' %page
        print(Hollys_url)
        html = urllib.request.urlopen(Hollys_url)
        soupHollys = BeautifulSoup(html, 'html.parser')
        tag_tbody = soupHollys.find('tbody')
        for store in tag_tbody.find_all('tr'):
            if len(store) <= 3:
                break
            store_td = store.find_all('td')
            store_name = store_td[1].string
            store_sido = store_td[0].string
            store_address = store_td[3].string
            store_phone = store_td[5].string
            result.append([store_name]+[store_sido]+[store_address]
                          +[store_phone])
            cnt += 1
            print(f'[{cnt:0>3}] {store_name}  \t {store_sido}')
            if cnt == MAX:
                MAX_flag = True
                break
        if MAX_flag:
            break

    return

#[CODE 0]
def main():
    result = []
    print('Hollys store crawling >>>>>>>>>>>>>>>>>>>>>>>>>>')
    hollys_store(result)   #[CODE 1] 호출
    hollys_tbl = pd.DataFrame(result, columns=('store', 'sido-gu', 'address','phone'))
    hollys_tbl.to_csv(FILE, encoding='cp949', mode='w', index=True)
    del result[:]
    return hollys_tbl


df = main()
df

#### **[실습] :  텍스트 데이터를 토큰화하여 빈도수를 구하고 빈도수에 따른 워드클라우드 만들기**
- 앞에서 수집한 할리스 커피매장의 매장 위치 정보 중 지역(**sido-gu**) 정보를 이용하여 워드 클라우드 만들기
- 필요한 라이브러리 및 주의사항
    - 사이킷런 CountVectorizer클래스 사용 : 단어 빈도수 추출
    - 단, 한글에서 불용어 처리 및 가중치 처리 등 자연어 텍스트 전처리에 필요한 여러 가지 방법은 여기서 다루지 않는다.

In [ ]:
# 사이킷런 설치하기
!pip install scikit-learn

In [ ]:
!pip install wordcloud

In [ ]:
import sklearn
sklearn.__version__

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10,6)

# 말뭉치를 토큰화하여 빈도수 가져오기
def get_wordTokenCount(corpus):
    from sklearn.feature_extraction.text import CountVectorizer

    # 말뭉치를 토큰화하기
    vect = CountVectorizer().fit(corpus)
    count = vect.transform(corpus).toarray().sum(axis=0)

    # 토큰 빈도수로 정렬하고 토큰명 추출
    idx = np.argsort(-count)  # 내림 정렬하여 인덱스 반환: 토큰의 인덱스
    count = count[idx]        # 토큰의 빈도수
    feature_name = np.array(vect.get_feature_names_out())[idx]  # 토큰값

    # 빈도수 많은 순서대로 토큰명 10개만 출력
    print(list(zip(feature_name, count))[:10])

    return feature_name, count


# 단어(토큰) 빈도수 막대 그래프 그리기
def draw_wordTokenCountGraph(data, freq):
    plt.bar(data, freq)
    plt.grid()
    plt.show()

    # 그래프 그림 저장히기
    plt.savefig(f'./token_bar_graph.png')

# 워드클라우드 만들기
def make_wordcloud(feature_name, count):
    # 한글 폰트 경로를 설정
    # font_path = 'NanumGothic'  #/usr/share/fonts/truetype/nanum/NanumGothic.ttf  #코랩
    font_path = 'malgun'  # C:/Windows/Fonts/                                  #window

    # (토큰명, 빈도수) 딕셔너리 타입으로 변환
    data = dict(zip(feature_name, count))

    # 워드클라우드로 그래프로 시각화
    wc = WordCloud(width = 1000, height = 600, background_color="black", font_path=font_path)
    plt.imshow(wc.generate_from_frequencies(data)) #딕셔너리
    plt.axis("off")
    plt.show()

    # 이미지 파일로 저장하기
    wc.to_file(f'./워드클라우드.png')


# 1.텍스트 말뭉치(corpus) 데이터 지정하기
corpus = df['sido-gu'].to_list()
# print(corpus)

# 2.말뭉치를 토큰화하여 빈도수 가져오기
feature_name, count = get_wordTokenCount(corpus)

# 3.단어(토큰) 빈도수 막대 그래프 그리기(상위 10개)
# draw_wordTokenCountGraph(feature_name[:10], count[:10])

# 3.워드 클라우드 만들기
make_wordcloud(feature_name, count)



---



### **[미션] : 네이버 뉴스 기사 제목을 워드클라우드로 만들기**
앞에서 정적크롤링으로 가져왔던 네이버 뉴스 기사 제목을 워드 클라우드로 만들어 보세요.

In [ ]:
import pandas as pd

df = pd.read_csv('./my_naver_news.csv')
df


# 1.텍스트 말뭉치(corpus) 데이터 지정하기
corpus = df['title'].to_list()
# print(corpus)

# 2.말뭉치를 토큰화하여 빈도수 가져오기
feature_name, count = get_wordTokenCount(corpus)

# 3.단어(토큰) 빈도수 막대 그래프 그리기(상위 10개)
# draw_wordTokenCountGraph(feature_name[:10], count[:10])

# 3.워드 클라우드 만들기
make_wordcloud(feature_name, count)



---



#### [실습문제] : 정적크롤링 방법을 이용하여 TEXT 정보 추출하여 표로 만들기
자신이 원하는 웹 페이지를 정해서 100개 이상 텍스트 정보 추출해서 표로 출력하기

----------